In [1]:
import random
import torch
import torchvision
from torch import nn

# **Load Fashion MNIST dataset**

In [2]:
def OneHotEncoding(labels):
    arr = []
    for label in labels:
        temp = [0 for i in range(0,10)]
        temp[label-1] = 1
        arr.append([temp])
    return torch.tensor(arr)

def GetTupple(X, y, index):
    return (X[index], y[index])

def GetList(X, y):
    temp, arr = X.size(), []
    temp = temp[0]
    for i in range(temp):
        arr.append(GetTupple(X, y, i))
    return arr

def Random(x):
    return random.randint(0, x-1)

In [3]:
training_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
    target_transform=torchvision.transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)
testing_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor(),
    target_transform=torchvision.transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
%%capture

X_train, y_train, X_test, y_test = training_data.train_data, training_data.train_labels, training_data.test_data, training_data.test_labels

X_train, X_test = X_train.numpy().reshape(60000, 1, 28, 28), X_test.numpy().reshape(60000, 1, 28, 28)
X_train, X_test = torch.from_numpy(X_train).type(dtype=torch.float), torch.from_numpy(X_test).type(dtype=torch.float)

# **Create our model**

In [37]:
from posixpath import join
class NeuralNetwork(nn.Module):
    def __init__(self, learning_rate=1e-3, batch_size=64, epochs=10):
        super(NeuralNetwork, self).__init__()
        
        # Hyper parameters
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs

        # Create layers
        self.Flatten = nn.Flatten()
        self.Linear_ReLU = nn.Sequential(
            nn.Linear(28*28, 512), nn.ReLU(),
            nn.Linear(512, 512), nn.ReLU(),
            nn.Linear(512, 10)
        )
        # self.Softmax = nn.Softmax()

        # Loss and optimizer
        self.loss = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
        
    def forward(self, x):
        x = self.Flatten(x)
        y = self.Linear_ReLU(x)
        # y = self.Softmax(y)
        return y
    
    def train(self, X_train, y_train):
        result = 0
        for index in range(0, X_train.size()[0] // self.batch_size):
            # index = [Random(X_train.size()[0]) for i in range(self.batch_size)]
            X = torch.tensor([(X_train[j]).numpy() for j in range(64*index, 64*index+64)], dtype=torch.float)
            y = torch.tensor([(y_train[j]).item() for j in range(64*index, 64*index+64)])

            y_pre = self.forward(X)
            result = self.loss(y_pre, y)
            self.optimizer.zero_grad()
            result.backward()
            self.optimizer.step()
        
        return result

    def fit(self, X_train, y_train):
        for i in range(self.epochs):
            loss = self.train(X_train, y_train)
            print(f"Epoch {i}: Loss = {loss}")
    
    def predict(self, X):
        return self.forward(X)

    def Evaluate(self, X, y):
        correct = 0
        with torch.no_grad():
            for index in range(0, X.size()[0]):
                y_pre = self.predict(X[index])
                if (y_pre.argmax().item() == y[index].item()):
                    correct += 1
        return correct / X.size()[0] * 100

# **Start training and testing this model**

In [38]:
model = NeuralNetwork(learning_rate=5e-4, batch_size=64, epochs=10)
model.fit(X_train, y_train)

Epoch 0: Loss = 0.30917447805404663
Epoch 1: Loss = 0.2629067897796631
Epoch 2: Loss = 0.24151629209518433
Epoch 3: Loss = 0.2265772968530655
Epoch 4: Loss = 0.21238471567630768
Epoch 5: Loss = 0.20128197968006134
Epoch 6: Loss = 0.1944742649793625
Epoch 7: Loss = 0.192623570561409
Epoch 8: Loss = 0.1800900399684906
Epoch 9: Loss = 0.17223826050758362


In [14]:
z = 100
print("Predicted label", z, " is", model.predict(X_train[z]).argmax(1))
print("Actual label", z, " is", y_train[z].item())

Predicted label 100  is tensor([8])
Actual label 100  is 8


In [39]:
model.Evaluate(X_train, y_train)

91.09333333333333